In [4]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
print("Libraries imported!")

Libraries imported!


In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

Matplotlib is building the font cache; this may take a moment.


In [6]:
# Load the data
df = pd.read_csv('Training_Data_16k.csv')
print("Data loaded!")

Data loaded!


In [7]:
# Display basic information
print("Dataset Shape:", df.shape)
print("\nFirst few rows:")
print(df.head())

print("\nColumn names:")
print(df.columns.tolist())

print("\nData types:")
print(df.dtypes)

print("\nMissing values:")
print(df.isnull().sum())

print("\nBasic statistics:")
print(df.describe())

Dataset Shape: (16000, 15)

First few rows:
   class  Aspect_30m.tif  Curvature_30m.tif  Dis2Roads_30m.tif  \
0      0       59.036243          -0.111111         247.386337   
1      0      315.000000           0.000000        1290.000000   
2      1      116.565048          -0.222222        7122.836426   
3      1       45.000000           0.444444        1018.675598   
4      1      123.690071          -0.555556        2621.011230   

   Dis2stream_30m.tif  Elevation_30m.tif  Geomorphology_30m.tif  \
0          510.881592                 43                      1   
1          918.313660                 45                      2   
2         2002.598267                 37                      3   
3          210.000000                 39                      2   
4          954.829834                 38                      1   

   Lithology_30m.tif  LULC_30m.tif  Rainfall_30m.tif  Slope_30m.tif  \
0                  2             5       1180.459351       7.307570   
1             